In [1]:
import urllib.request
import pandas as pd

In [9]:
!pip install torchtext

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x230ffc8ffa0>)

In [3]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')

In [4]:
train_df = df[:25000]
test_df = df[25000:]

In [21]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [14]:
from torchtext.legacy import data  # torchtext.data 불러오기

In [15]:
# 필드 정의
TEXT = data.Field(sequential=True, # 시퀀스 데이터 여부
                  use_vocab=True, # 단어 집합을 만들 것인지 여부
                  tokenize=str.split, # 토크나이저를 따로 사용하지 않고 split 사용
                  lower=True, # 소문자화 할 것인지 여부
                  batch_first=True, # 미니 배치를 앞으로 하여 데이터 불러오기
                  fix_length=20) # 최대 허용 길이 / 이에 맞춰 패딩 작업

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True) # 레이블 데이터 여부

## 데이터셋 만들기

In [17]:
from torchtext.legacy.data import TabularDataset # 필드에서 정의 했던 토큰화 방법으로 토큰화 수행
from torchtext.legacy.data import Iterator

In [22]:
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True) # skip_header : 데이터의 첫본째 줄은 무시

In [23]:
# 단어집합 만들기
TEXT.build_vocab(train_data, min_freq=10, max_size=10000) # 단어집합 크기 설정

In [24]:
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) # 첫번째 미니배치를 받아와서 batch라는 변수에 저장

In [25]:
print(batch.text) # (배치크기 * fix_length)

tensor([[   9,   39,  260,    2,  782,    0,  752,    0,   19,    2,    0, 2608,
          269,    0,   11,    7,  192,    3, 4501,   30],
        [   9, 1969,  328,   11,   18,    9,  281,    6,   37,  127,   12,   23,
          950,    6,   28,  910,    4,   23,   39,  971],
        [  43,   49,  705,    6, 8066,   12, 3540, 1442,  992,    0,   10, 1158,
            7,    0,   16,    2, 1014,    0,    2,  277],
        [  82,    5,  381,   10,   20, 1430,   88,    5,    2,  174,  125,    9,
          307,    6,   26,    6,  109,    8, 3733,  913],
        [4646,  550,    0,   40,    0,    9,  183, 1025, 1913,   16,  550,    0,
           22, 2273,   21,    0, 2781,    5,  102, 4734]])


## batch_first = False로 하였을 경우의 텐서 크기

In [30]:
# 필드 재정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

In [33]:
train_data, test_data = TabularDataset.splits(
        path='.', train='train_data.csv', test='test_data.csv', format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

In [34]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [35]:
batch_size = 5
train_loader = Iterator(dataset=train_data, batch_size = batch_size)
batch = next(iter(train_loader)) 

In [36]:
print(batch.text)

tensor([[   9,    9,   43,   82, 4646],
        [  39, 1969,   49,    5,  550],
        [ 260,  328,  705,  381,    0],
        [   2,   11,    6,   10,   40],
        [ 782,   18, 8066,   20,    0],
        [   0,    9,   12, 1430,    9],
        [ 752,  281, 3540,   88,  183],
        [   0,    6, 1442,    5, 1025],
        [  19,   37,  992,    2, 1913],
        [   2,  127,    0,  174,   16],
        [   0,   12,   10,  125,  550],
        [2608,   23, 1158,    9,    0],
        [ 269,  950,    7,  307,   22],
        [   0,    6,    0,    6, 2273],
        [  11,   28,   16,   26,   21],
        [   7,  910,    2,    6,    0],
        [ 192,    4, 1014,  109, 2781],
        [   3,   23,    0,    8,    5],
        [4501,   39,    2, 3733,  102],
        [  30,  971,  277,  913, 4734]])


In [37]:
print(batch.text.shape)

torch.Size([20, 5])
